In [ ]:
import pathlib
import sys
import os
import platform

import requests

import pandas as pd
import numpy as np
import geopandas as gpd
#import contextily as cx

import seaborn as sns

import time
import datetime

import statsmodels.api as sm

import logging

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib
import matplotlib as mpl
import matplotlib.cm as cm

#%matplotlib notebook
#%matplotlib widget
import matplotlib.animation as animation
from matplotlib.dates import DateFormatter

%matplotlib inline

In [ ]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Numpy Version " + str(np.__version__))
print (os.environ['CONDA_DEFAULT_ENV'])

In [ ]:






#plt.twinx()
#oet_b28['Ensemble ET'].plot(color='red')
#oet_b28['Precip (gridMET)'].plot(color='green')

oet = {}
flows = {}

site_ids = ['BC22','BC28']

timesteps = ['2023-03-18','2023-04-07','2023-04-17','2023-04-27','2023-05-02','2023-05-22',
             '2023-06-26','2023-07-01','2023-07-06','2023-07-11','2023-07-16','2023-07-21',
             '2023-08-05','2023-08-15','2023-08-30','2023-09-19','2023-09-24','2023-09-29',
             '2023-10-04','2023-10-09','2023-10-19','2023-10-24','2023-10-29','2023-11-08','2023-11-28']
timesteps = pd.to_datetime(timesteps)

fig, ax = plt.subplots(2,1,sharex=True)

ax2 = {}
line1 = {}
line2 = {}
line3 = {}
label = {}


i=0
for site_id in site_ids:

    flows[site_id]= pd.read_parquet(ctnpath / f"{site_id}.parquet")
    
    oet[site_id] = pd.read_csv(ctnpath / f"openet_{site_id}.csv",parse_dates=True,index_col='DateTime')

    if i == 0:
        lab1 = 'OpenET ens ET'
        lab2 = 'Gridmet Precip'
        lab3 = 'Diversion'
    else:
        lab1 = None
        lab2 = None
        lab3 = None
    line1[i], = ax[i].plot(oet[site_id].index[0], oet[site_id].iloc[0]['Ensemble ET'],label=lab1,color='red',zorder=1)
    line2[i], = ax[i].plot(oet[site_id].index[0], oet[site_id].iloc[0]['Precip (gridMET)'],label=lab2,color='green',alpha=0.5,zorder=2)
    
    ax2[i] = ax[i].twinx()
    line3[i], = ax2[i].plot(flows[site_id].index[0],flows[site_id].iloc[0]['acft'],color='blue',alpha=0.5,zorder=10,label=lab3)

    ax[i].set_title(site_id)
    
    
    ax2[i].set_ylim(0,5)
    ax2[i].set_ylabel("Pumping (acft/day)")
    ax[i].set_ylabel("Precip and ET (in/day)")
    i+=1
    ## Rotate date labels automatically
    #fig.autofmt_xdate()
myFmt = DateFormatter("%b")
label = ax[0].text(pd.to_datetime('2023-09-20'),0.5, f"{timesteps[0]:%Y-%m-%d}",fontsize=12,
                     color="black",zorder=0)
fig.legend(loc='outside upper center',ncols=3)
ax[1].xaxis.set_major_formatter(myFmt)
ax[1].set_xlim(pd.to_datetime("2023-01-01"),pd.to_datetime("2023-12-01"))
ax[1].set_xlabel('Month')


def update(frame):
    # for each frame, update the data stored on each artist.
    for j in [0,1]:
        line1[j].set_xdata(oet[site_ids[j]].loc[:timesteps[frame]].index)
        line1[j].set_ydata(oet[site_ids[j]].loc[:timesteps[frame],'Ensemble ET'])
        line2[j].set_xdata(oet[site_ids[j]].loc[:timesteps[frame]].index)
        line2[j].set_ydata(oet[site_ids[j]].loc[:timesteps[frame],'Precip (gridMET)'])
        line3[j].set_xdata(flows[site_ids[j]].loc[:timesteps[frame]].index)
        line3[j].set_ydata(flows[site_ids[j]].loc[:timesteps[frame],'acft'])
    label.set_text(f"{timesteps[frame]:%Y-%m-%d}")
    return (line1[0],line1[1], line2[0],line2[1], line3[0],line3[1], label)


anim = animation.FuncAnimation(fig=fig, func=update, frames=len(timesteps), interval=1000)

anim.save(filename= ctnpath / "twosites.gif", writer="pillow")
plt.show()